# Teste completo: Baseline CNN vs Híbrido CNN+VQC (Colab + GPU)

Este notebook roda no **Google Colab** com GPU para replicar a comparação do artigo (Pereira & Saraiva, 2021):
- **Baseline:** 3 runs × 100 épocas, cenários No Balance e ROS (altere N_RUNS_* na célula 5 para 10 se quiser replicar o artigo).
- **Híbrido:** mesmo número de runs.

**Antes de rodar:** ative o runtime com GPU (Runtime → Change runtime type → GPU).

### O que voce pode alterar no notebook

**Rodar localmente:** Ative o venv (`source .venv/bin/activate`), instale com `pip install -r requirements.txt` na raiz do projeto, e abra o Jupyter na raiz ou em `notebooks/`. O notebook detecta a raiz e ajusta o path; o dataset vem do kagglehub ou defina `SGCC_DATASET_PATH`.

1. **Celula de clone/setup:** Em Colab, clona o repo; localmente so define a raiz do projeto.

2. **Credenciais (Colab/Kaggle):** Colab = Secrets (KAGGLE_USERNAME, KAGGLE_KEY). Local = `~/.kaggle/kaggle.json` ou `SGCC_DATASET_PATH` para a pasta do dataset.

3. **Celula “Carregar dados e parametros”:**
- **N_RUNS_BASELINE** e **N_RUNS_HYBRID** = 3. Use 10 para replicar o artigo.
- **EPOCHS** = 100. Reduza (ex: 5) para teste rapido.
- **SEEDS** = [0, 42, 123] para variedade por run.
- **Dataset:** Colab/Kaggle ou env `SGCC_DATASET_PATH`; senao o notebook tenta baixar via kagglehub.

## 1. Configurar ambiente e clonar o repositório

### Tempo estimado e como nao desligar / nao desconectar

**Tempo aproximado:** Com **3 runs × 2 cenários = 6 runs** por modelo (100 épocas):
- **Baseline (GPU):** ~2–5 min por run → **~15–30 min** no total.
- **Híbrido (CNN+VQC):** O VQC em si não é “lento”; o que pesa é **simular o circuito quântico na CPU** (PennyLane). Por run pode levar **~15–45 min** → com 3 runs, **~1h30–2h30** no total. Com 10 runs seria bem mais longo.

Para teste ainda mais rápido: use menos épocas (ex: 5).

**Para o PC nao desligar (rodando localmente no Linux):**
- **Desativar suspensao:** Configuracoes → Energia → “Quando inativo” = Nunca (ou apenas desligar tela).
- **Via terminal:** `systemctl mask sleep.target suspend.target hibernate.target hybrid-sleep.target` (reverter depois com `unmask`).
- **Impedir que a tela bloqueie so durante o treino:** use `caffeinate` (macOS) ou no Linux algo como `xset s off; xset -dpms` antes de rodar (reverte depois com `xset s on; xset +dpms`), ou instale “Caffeine” / “Inhibit” no seu ambiente de desktop.

**No Google Colab:**
- O Colab pode desconectar apos ~90 min de inatividade. Mantenha a aba aberta e, se quiser, use um “keep-alive” no navegador (extensoes que simulam atividade) ou um script que rode em loop no console (ex: clicar no codigo a cada alguns minutos). Nao ha garantia; para runs muito longos, prefira rodar por partes (menos runs por vez) ou salvar checkpoints.

In [ ]:
# Verificar GPU (TensorFlow usa automaticamente no Colab)
import tensorflow as tf
print("GPU disponível:", tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)

In [ ]:
# Colab/Kaggle: clonar e entrar no repo. Local: usar raiz do projeto (notebooks/ ou repo).
import os

def _project_root():
    cwd = os.path.abspath(os.getcwd())
    for d in [cwd, os.path.join(cwd, ".."), os.path.join(cwd, "..", "..")]:
        d = os.path.abspath(d)
        if os.path.isdir(os.path.join(d, "src")) and os.path.isfile(os.path.join(d, "train_baseline_pereira.py")):
            return d
    return cwd

try:
    import google.colab
    _in_colab = True
except ImportError:
    _in_colab = False
_in_kaggle = bool(os.environ.get("KAGGLE_KERNEL_RUN_TYPE"))

if _in_colab and not os.path.isdir("src"):
    get_ipython().system("git clone https://github.com/alanveloso/electricity-theft-hybrid-qml.git 2>/dev/null || true")
    get_ipython().run_line_magic("cd", "electricity-theft-hybrid-qml")
    print("Repositório clonado (Colab/Kaggle).")
else:
    root = _project_root()
    if root != os.getcwd():
        os.chdir(root)
        print("Raiz do projeto (local):", root)
    else:
        print("Diretório atual:", os.getcwd())

In [ ]:
# Instalar dependências: Colab/Kaggle roda pip; local use o venv (pip install -r requirements.txt).
if _in_colab or _in_kaggle:
    get_ipython().system("pip install -q numpy pandas scikit-learn tensorflow kagglehub pennylane pennylane-lightning matplotlib")
else:
    print("Ambiente local: use 'pip install -r requirements.txt' na raiz do projeto.")

## 2. Configurar Kaggle (dataset SGCC)

Opção A: Colab Secrets - em Key adicione KAGGLE_USERNAME e KAGGLE_KEY (ou KAGGLE_API_TOKEN).
Opção B: Faça upload do kaggle.json (Account, Create New Token no Kaggle) na célula abaixo.

In [ ]:
import os

# Kaggle: credenciais já disponíveis. Colab: usar Secrets. Local: kagglehub usa ~/.kaggle/kaggle.json ou env.
try:
    from google.colab import userdata
    if os.environ.get("KAGGLE_KEY") is None:
        os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")
    if os.environ.get("KAGGLE_USERNAME") is None:
        os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
    print("Credenciais Kaggle carregadas dos Secrets (Colab).")
except Exception as e:
    if os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
        print("Kaggle: credenciais do ambiente.")
    else:
        print("Secrets não configurados ou rodando localmente:", e)
        print("Local: configure ~/.kaggle/kaggle.json ou defina SGCC_DATASET_PATH para a pasta do dataset.")

## 3. Carregar dados e parâmetros do teste completo

In [ ]:
import sys
import os

# Garantir raiz do projeto no path e no cwd (funciona em Colab/Kaggle e local)
def _root():
    cwd = os.path.abspath(os.getcwd())
    for d in [cwd, os.path.join(cwd, ".."), os.path.join(cwd, "..", "..")]:
        d = os.path.abspath(d)
        if os.path.isdir(os.path.join(d, "src")) and os.path.isfile(os.path.join(d, "train_baseline_pereira.py")):
            return d
    return cwd

ROOT = _root()
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)
if os.getcwd() != ROOT:
    os.chdir(ROOT)

from src.data.sgcc_loader import load_sgcc_from_path, train_test_split
from train_baseline_pereira import get_dataset_path

# Modo local: teste rápido (1 run, 2 épocas) para validar o fluxo. Colab/Kaggle: use False.
try:
    _run_local = not (_in_colab or _in_kaggle)
except NameError:
    _run_local = True  # se rodou a célula sem as anteriores
QUICK_TEST = _run_local  # True = 1 run × 2 épocas; False = 3 runs × 100 épocas

# Parâmetros do teste completo (como no artigo)
N_RUNS_BASELINE = 1 if QUICK_TEST else 3   # 10 no artigo
N_RUNS_HYBRID = 1 if QUICK_TEST else 3
EPOCHS = 2 if QUICK_TEST else 100
SEED = 42
SEEDS = [0, 42, 123]
VERBOSE = 1 if QUICK_TEST else 0  # 1 no quick test para ver progresso
if QUICK_TEST:
    print("Modo QUICK_TEST: 1 run × 2 épocas por cenário (para validar localmente).")

path = get_dataset_path(None)
print("Carregando dataset SGCC...")
X, y = load_sgcc_from_path(path, seed=SEED, preprocessing="pereira")
print(f"X.shape = {X.shape}, y.shape = {y.shape}")
print(f"Normal: {(y==0).sum()}, Fraude: {(y==1).sum()}")

### Verificar se dados e modelo estão carregados corretamente

Rode a célula abaixo para checar: **dados** (shape, classes, sem NaN) e **modelo baseline** (uma predição). Assim você confirma que o pipeline está certo antes do treino longo. (Para verificação completa incluindo híbrido, use `python scripts/verify_model_loading.py` na raiz do projeto.)

In [ ]:
# Verificação rápida: dados + modelo baseline
import numpy as np
from src.models.cnn import build_cnn

# 1) Dados
print("=== Dados (X, y) ===")
print(f"  X.shape = {X.shape}  (esperado: (n, 148, 7, 1))")
print(f"  y.shape = {y.shape}, classes: 0={(y==0).sum()}, 1={(y==1).sum()}")
print(f"  X sem NaN: {not np.any(np.isnan(X))}")
ok_data = X.ndim == 4 and X.shape[1:] == (148, 7, 1) and y.shape[0] == X.shape[0]
print(f"  Formato OK: {ok_data}\n")

# 2) Modelo baseline: construir e uma predição
print("=== Modelo baseline (CNN) ===")
model = build_cnn(input_shape=(148, 7, 1))
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"])
sample = X[:2]
out = model.predict(sample, verbose=0)
ok_model = out.shape == (2, 2) and np.allclose(out.sum(axis=1), 1.0)
print(f"  Entrada: {sample.shape} -> Saída: {out.shape} (esperado (2, 2))")
print(f"  Modelo carrega e prediz OK: {ok_model}")
print("\nTudo OK para treino." if (ok_data and ok_model) else "Revise dados ou modelo.")

## 4. Rodar Baseline (CNN) — 3 runs × 100 épocas

In [ ]:
from train_baseline_pereira import run_scenario as run_baseline_scenario
import numpy as np

results_baseline = []
for scenario in ["no_balance", "ros"]:
    aucs, accs, times = [], [], []
    for run in range(N_RUNS_BASELINE):
        run_seed = SEEDS[run % len(SEEDS)]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, stratify=True, seed=run_seed
        )
        # class_weight=True em no_balance evita colapso para a classe majoritária
        res, _, _ = run_baseline_scenario(
            scenario, X_train, y_train, X_test, y_test,
            epochs=EPOCHS, verbose=VERBOSE, learning_rate=0.01, use_class_weight=True,
            run_name=f"baseline_{scenario}_run{run+1}", out_dir="."
        )
        aucs.append(res["auc"])
        accs.append(res["accuracy"])
        times.append(res["train_time_seconds"])
        auc_str = f"{res['auc']:.4f}" if not np.isnan(res["auc"]) else "nan"
        print(f"  Baseline {scenario} run {run+1}/{N_RUNS_BASELINE} - AUC: {auc_str}, Acc: {res['accuracy']*100:.2f}%, Tempo: {res['train_time_seconds']:.0f}s")
    mean_auc = np.nanmean(aucs)
    std_auc = np.nanstd(aucs, ddof=1) if len(aucs) > 1 else 0
    if np.isnan(std_auc):
        std_auc = 0.0
    mean_acc = np.mean(accs) * 100
    total_time = sum(times)
    mean_time = np.mean(times)
    results_baseline.append({"scenario": scenario, "mean_auc": mean_auc, "std_auc": std_auc, "mean_acc": mean_acc, "total_time_s": total_time, "mean_time_s": mean_time})
    mean_auc_str = f"{mean_auc:.4f}" if not np.isnan(mean_auc) else "nan"
    print(f"  -> Media AUC: {mean_auc_str} +- {std_auc:.4f}, Media Acc: {mean_acc:.2f}% | Tempo total: {total_time:.0f}s ({mean_time:.0f}s/run)")
print("Baseline concluido.")

## 5. Rodar Híbrido (CNN+VQC) — 3 runs × 100 épocas

*(Mesmo número de runs do baseline para comparação justa. O circuito quântico é simulado em CPU; o treino da CNN usa GPU.)*

**Execução em computador IBM:** defina no ambiente `QML_DEVICE=qiskit.ibmq`, `QML_IBMQ_BACKEND=...` e `IBMQX_TOKEN=...`. O código configura e valida o device antes do treino (se falhar, o erro aparece logo).

In [ ]:
from train_hybrid_pereira import run_hybrid_scenario

results_hybrid = []
for scenario in ["no_balance", "ros"]:
    aucs, accs, times = [], [], []
    for run in range(N_RUNS_HYBRID):
        run_seed = SEEDS[run % len(SEEDS)]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, stratify=True, seed=run_seed
        )
        # use_class_weight=True (default) em no_balance; run_name para salvar checkpoint no Kaggle
        res = run_hybrid_scenario(
            scenario, X_train, y_train, X_test, y_test,
            epochs=EPOCHS, verbose=VERBOSE, learning_rate=0.01,
            run_name=f"hybrid_{scenario}_run{run+1}", out_dir="."
        )
        aucs.append(res["auc"])
        accs.append(res["accuracy"])
        times.append(res["train_time_seconds"])
        auc_str = f"{res['auc']:.4f}" if not np.isnan(res["auc"]) else "nan"
        print(f"  Hibrido {scenario} run {run+1}/{N_RUNS_HYBRID} - AUC: {auc_str}, Acc: {res['accuracy']*100:.2f}%, Tempo: {res['train_time_seconds']:.0f}s")
    mean_auc = np.nanmean(aucs)
    std_auc = np.nanstd(aucs, ddof=1) if len(aucs) > 1 else 0
    if np.isnan(std_auc):
        std_auc = 0.0
    mean_acc = np.mean(accs) * 100
    total_time = sum(times)
    mean_time = np.mean(times)
    results_hybrid.append({"scenario": scenario, "mean_auc": mean_auc, "std_auc": std_auc, "mean_acc": mean_acc, "total_time_s": total_time, "mean_time_s": mean_time})
    mean_auc_str = f"{mean_auc:.4f}" if not np.isnan(mean_auc) else "nan"
    print(f"  -> Media AUC: {mean_auc_str} +- {std_auc:.4f}, Media Acc: {mean_acc:.2f}% | Tempo total: {total_time:.0f}s ({mean_time:.0f}s/run)")
print("Hibrido concluido.")

## 6. Tabela de comparação e referência do artigo

In [ ]:
import pandas as pd

def _auc_str(mean_auc, std_auc):
    """Formata AUC para tabela; usa 'nan (colapso?)' quando AUC indefinida."""
    if np.isnan(mean_auc):
        return "nan (colapso?)"
    return f"{mean_auc:.4f} +- {std_auc:.4f}"

rows = []
for r in results_baseline:
    rows.append({
        "Modelo": "Baseline (CNN)",
        "Cenario": r["scenario"],
        "Media AUC": _auc_str(r["mean_auc"], r["std_auc"]),
        "Media Acuracia (%)": f"{r['mean_acc']:.2f}",
        "Runs": N_RUNS_BASELINE,
        "Tempo total (s)": round(r["total_time_s"], 0),
        "Tempo medio/run (s)": round(r["mean_time_s"], 0)
    })
for r in results_hybrid:
    rows.append({
        "Modelo": "Hibrido (CNN+VQC)",
        "Cenario": r["scenario"],
        "Media AUC": _auc_str(r["mean_auc"], r["std_auc"]),
        "Media Acuracia (%)": f"{r['mean_acc']:.2f}",
        "Runs": N_RUNS_HYBRID,
        "Tempo total (s)": round(r["total_time_s"], 0),
        "Tempo medio/run (s)": round(r["mean_time_s"], 0)
    })

df = pd.DataFrame(rows)
display(df)

total_baseline = sum(r["total_time_s"] for r in results_baseline)
total_hybrid = sum(r["total_time_s"] for r in results_hybrid)
print(f"Tempo total Baseline: {total_baseline/60:.1f} min | Tempo total Hibrido: {total_hybrid/60:.1f} min")
print("Referencia (Pereira & Saraiva 2021, 10 runs, 100 epocas):")
print("  No Balance: AUC 0,5162 +- 0,0045   Acuracia 91,59%")
print("  ROS:        AUC 0,6714 +- 0,0062   Acuracia 67,78%")

In [ ]:
# Salvar resultados em CSV (download opcional)
df.to_csv("colab_full_test_results.csv", index=False)
print("Resultados salvos em colab_full_test_results.csv")